In [ ]:
!pip install -qU langchain-huggingface
!pip install -qU langchain-qdrant
!pip install -q langchain
!pip install -q langchain-community
!pip install -U langchain-text-splitter
!pip install -U redis
!pip install -U langchain-groq
!pip install -U chromadb
!pip install --upgrade --quiet  redis redisvl langchain-openai tiktoken lark

In [1]:

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.docstore.document import Document
from typing import List, Dict
from langchain.vectorstores import Redis
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import os

In [2]:


# Function to load and chunk data from a folder
def load_and_chunk_data(data_path):
    docs = []
    # Load all .txt files from the specified folder
    for filename in os.listdir(data_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(data_path, filename)
            loader = TextLoader(file_path, encoding='utf-8')
            docs.extend(loader.load())

    # Define headers to split on for Markdown splitting
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    # Initialize Markdown splitter
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )

    chunk_size = 512
    chunk_overlap = 0

    # Initialize character-based splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    chunked_docs = []

    # Process each document
    for doc in docs:
        # Split document by Markdown headers
        md_header_splits = markdown_splitter.split_text(doc.page_content)

        # Further split the Markdown header splits into smaller chunks
        chunked_docs.extend(text_splitter.split_documents(md_header_splits))

    return chunked_docs


data_path = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/thongtintuyensinh'
chunked_data = load_and_chunk_data(data_path)


In [3]:
def build_index_schema(documents: List[Document]) -> Dict:
    schema = {"text": []}
    for doc in documents:
        for key in doc.metadata:
            name_dict = {"name": f"{key}"}
            if name_dict not in schema["text"]:
                schema["text"].append(name_dict)
    return schema

index_schema = build_index_schema(chunked_data)
print(index_schema)

{'text': [{'name': 'Header 1'}, {'name': 'Header 2'}, {'name': 'Header 3'}]}


In [4]:

from dotenv import load_dotenv
load_dotenv()
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1,api_key= os.getenv('llm_api_1'))
HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1,api_key= os.getenv('llm_api_1'))


def print_result(response_obj):
    print("SOURCES: \n")
    cnt = 1
    for source_doc in response_obj["source_documents"]:
        print(f"Chunk #{cnt}")
        cnt += 1
        print("Source Metadata: ", source_doc.metadata)
        print("Source Text:")
        print(source_doc.page_content)
        print("\n")
    print("RESULT: \n")
    print(response_obj["result"] + "\n\n")


vectorstore = Redis.from_documents(
    chunked_data,
    HF_EMBEDDING,
    redis_url='redis://localhost:6379',
    index_schema = index_schema
)
# retriever = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstore.as_retriever(search_kwargs={"k": 10}),
#     return_source_documents=True
# )

# query = "Thi khối nào để vào học viện kỹ thuật quân sự"
# response = retriever({"query": query})
# print_result(response)

/home/justtuananh/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/justtuananh/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`index_schema` does not match generated metadata schema.
If you meant to manually override the schema, please ignore this message.
index_schema: {'text': [{'name': 'Header 1'}, {'name': 'Header 2'}, {'name': 'Header 3'}]}
generated_schema: {'text': [{'name': 'Header 1'}, {'name': 'Header 2'}], 'numeric': [], 'tag': []}



In [5]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
retriever.invoke("tao thi đại học được 26 điểm khối C thì có đủ điều kiện xét tuyển vào học viện kĩ thuật quân sự không ?")

Metadata key Header 3 not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['Header 1', 'Header 2', 'Header 3']
Metadata key Header 3 not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['Header 1', 'Header 2', 'Header 3']
Metadata key Header 3 not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['Header 1', 'Header 2', 'Header 3']


[Document(metadata={'id': 'doc:d7707b5549f14421a6bce9194ce5c366:b450df5f2ded4150a322b152dcd935ae', 'Header 1': 'THÔNG TIN KỲ TUYỂN SINH CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024', 'Header 2': '8\\. Tổ chức xét tuyển', 'Header 3': '8.2. Phương thức 2: Ưu tiên xét tuyển**'}, page_content='\\- Thí sinh đạt giải Tư cuộc thi khoa học, kỹ thuật cấp Quốc gia do Bộ GD&ĐT tổ chức, có nội dung đề tài dự thi phù hợp với ngành đào tạo của Học viện KTQS, tốt nghiệp THPT năm 2024, thời gian đạt giải không quá 03 năm tính tới thời điểm xét tuyển.  \nb) Hồ sơ đăng ký Ưu tiên xét tuyển  \n\\- Hồ sơ đăng ký Ưu tiên xét tuyển gồm:  \n\\+ Phiếu đăng ký ưu tiên xét tuyển (Theo mẫu của Bộ Quốc phòng).'),
 Document(metadata={'id': 'doc:d7707b5549f14421a6bce9194ce5c366:2717dff9bef8439e8b3954b03c32c7fe', 'Header 1': 'CƠ HỘI HỌC TẬP VÀ ĐIỂM MỚI TUYỂN SINH NĂM 2024 CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ', 'Header 2': 'II. MỘT SỐ ĐIỂM MỚI TRONG CÔNG TÁC TUYỂN SINH TẠI HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024**', 'Header 3': None},

In [6]:
# add self-query
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
    AttributeInfo)

document_content_description = "The document contains introductory information about the admissions details of the Military Technical Academy."

headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]


def generate_metadata_desc(field_name: str, header_info: List[tuple], qa_retriever: RetrievalQA) -> str:
    query = f"""
    Given a list of tuples indicating all possible metadata fields for this article,
    provide a very brief description (15 words or less) of the specified field,
    including its position in the hierarchy of headers (Header 1, Header 2, Header 3, etc.).

    all fields {header_info}
    specified field: {field_name}
    """

    return qa_retriever({"query": query})["result"]

def build_metadata_field_info(schema: Dict, header_info: List[tuple], qa_retriever: RetrievalQA) -> List[AttributeInfo]:
    filter_instructions = \
        "ALWAYS filter with one or more CONTAINS comparators, and use the OR operator to check ALL other fields."\
        "if the value of this field contains a word or phrase that is very similar to a word or phrase in the query, " \
        "filter for the exact string from the value rather than the query."
    filter_h1 = \
        "if the query contains a misspelling or an alternate spelling of this name, "\
        "filter for the value of this field rather than the name in the query."\
        "the h1-level filter should ALWAYS be combined with subsection filters using an AND operator. \n"
    filter_exclusion = \
        " NEVER filter this field on the value of 'Header 1'."
    attr_info_list = []
    for field in schema["text"]:
        desc = generate_metadata_desc(field["name"], header_info, qa_retriever) + filter_instructions
        if field["name"] == "Header 1":
            new_attr = AttributeInfo(
                name=field["name"],
                description = filter_h1 + desc,
                type="string"
            )
        else:
            new_attr = AttributeInfo(
                name=field["name"],
                description = desc + filter_exclusion,
                type="string"
            )
        attr_info_list.append(new_attr)
    return attr_info_list

metadata_field_info = build_metadata_field_info(index_schema, headers_to_split_on, retriever)

TypeError: 'RedisVectorStoreRetriever' object is not callable